In [1]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

## memory 공유 확인 위한 dummy chain 생성

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

dummy_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
dummy_prompt.pretty_print()

================================ System Message ================================


            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

input:{input}
order_id:{order_id}


In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [4]:
dummy_chain = dummy_prompt | model 

In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory

dumy_chain_with_memory = RunnableWithMessageHistory(
    dummy_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [6]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 변경하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240517-3"}}
    )
response

Parent run e44217c7-7a2c-47b6-bc4e-749c1d867814 not found for run d31b156d-3079-4b03-a065-61f3f34fbdb7. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-22783fa0-2a50-4280-bd25-597866a41c1e-0')

In [7]:
store["test_240517-3"].messages

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-22783fa0-2a50-4280-bd25-597866a41c1e-0')]

## 주문 변경/취소 분류 체인

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.

            Classify as '주문 변경' if there are any indications or keywords related to modifying an order, such as:
            - 변경
            - 수정
            - 바꾸다
            - 변경하고 싶어

            Classify as '주문 취소' if there are any indications or keywords related to canceling an order, such as:
            - 취소
            - 취소하고 싶어
            - 취소 요청
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
order_change_cancel_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n\n            Classify as '주문 변경' if there are any indications or keywords related to modifying an order, such as:\n            - 변경\n            - 수정\n            - 바꾸다\n            - 변경하고 싶어\n\n            Classify as '주문 취소' if there are any indications or keywords related to can

In [9]:
classify_change_or_cancel_chain = order_change_cancel_prompt | model
classify_change_or_cancel_chain

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n\n            Classify as '주문 변경' if there are any indications or keywords related to modifying an order, such as:\n            - 변경\n            - 수정\n            - 바꾸다\n            - 변경하고 싶어\n\n            Classify as '주문 취소' if there are any indications or keywords related to can

In [10]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_change_or_cancel_chain_with_memory = RunnableWithMessageHistory(
    classify_change_or_cancel_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [11]:
from langchain_core.runnables import RunnableLambda
classify_change_or_cancel_chain_with_memory_lambda = RunnableLambda(
    lambda x: classify_change_or_cancel_chain_with_memory.invoke(x,
                                                                 config={"configurable": {"session_id": "test_240517-3"}})
)
classify_change_or_cancel_chain_with_memory_lambda

RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))

## 승인 메시지 식별 체인

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

classify_confirmation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.
            
            To make this determination, the following conditions must be met:
            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.
            2. The user must have explicitly expressed consent in response to the AIMessage asking for approval of the action specified in action_type.
            
            Your response must be either 'yes' or 'no'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}\naction_type: {action_type}"),
    ]
)
classify_confirmation_prompt

ChatPromptTemplate(input_variables=['action_type', 'chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.\n            \n            To make this determination, the following conditions must be met:\n            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.\n            2. The user m

In [13]:
classify_confirmation_chain = classify_confirmation_prompt | model 
classify_confirmation_chain

ChatPromptTemplate(input_variables=['action_type', 'chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.\n            \n            To make this determination, the following conditions must be met:\n            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.\n            2. The user m

In [14]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_confirmation_chain_with_memory = RunnableWithMessageHistory(
    classify_confirmation_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [15]:
from langchain_core.runnables import RunnableLambda

classify_confirmation_chain_with_memory_lambda = RunnableLambda(
    lambda x: classify_confirmation_chain_with_memory.invoke(x,
                                                             config={"configurable": {"session_id": "test_240517-3"}})
)
classify_confirmation_chain_with_memory_lambda

RunnableLambda(lambda x: classify_confirmation_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))

## action_type 추가 위한 보조 함수

In [16]:
def helper(dict):
    inputs = dict["inputs"]
    action_type =  dict["action_type"]
    # action_type = action_type.content
    inputs["action_type"] = action_type
    return inputs

## 종합-입력에 승인 메시지 포함 여부까지 추가

In [17]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

confirmation_chain = (
    {
        "inputs": RunnablePassthrough(),
        "action_type": classify_change_or_cancel_chain_with_memory_lambda
    } 
    | RunnableLambda(helper) 
    | RunnablePassthrough.assign(execution_confirmation=classify_confirmation_chain_with_memory_lambda)
    )
confirmation_chain

{
  inputs: RunnablePassthrough(),
  action_type: RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))
}
| RunnableLambda(helper)
| RunnableAssign(mapper={
    execution_confirmation: RunnableLambda(lambda x: classify_confirmation_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))
  })

## 승인 메시지 작성 체인

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

generate_confirm_message_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that generates a confirmation request message based on the action_type and queried_result.
            create a message to show the queried_result and ask for final confirmation considering action_type.
            The response should be generated in Korean.
            """
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("human", "action_type:{action_type}\nqueried_result:{queried_result}")
    ]
)
generate_confirm_message_prompt

ChatPromptTemplate(input_variables=['action_type', 'queried_result'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            You are a robot that generates a confirmation request message based on the action_type and queried_result.\n            create a message to show the queried_result and ask for final confirmation considering action_type.\n            The response should be generated in Korean.\n            ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['action_type', 'queried_result'], template='action_type:{action_type}\nqueried_result:{queried_result}'))])

In [19]:
generate_confirm_message_chain = generate_confirm_message_prompt | model
generate_confirm_message_chain

ChatPromptTemplate(input_variables=['action_type', 'queried_result'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            You are a robot that generates a confirmation request message based on the action_type and queried_result.\n            create a message to show the queried_result and ask for final confirmation considering action_type.\n            The response should be generated in Korean.\n            ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['action_type', 'queried_result'], template='action_type:{action_type}\nqueried_result:{queried_result}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002B823EC3010>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002B823EFC790>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [20]:
from langchain_core.runnables.history import RunnableWithMessageHistory

generate_confirm_message_chain_with_memory = RunnableWithMessageHistory(
    generate_confirm_message_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [21]:
from langchain_core.runnables import RunnableLambda
generate_confirm_message_chain_with_memory_lambda = RunnableLambda(
    lambda x: generate_confirm_message_chain_with_memory.invoke(x,
                                                             config={"configurable": {"session_id": "test_240517-3"}})
)
generate_confirm_message_chain_with_memory_lambda

RunnableLambda(lambda x: generate_confirm_message_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))

## 주문 조회 함수

In [22]:
def fetch_recent_orders(dict):
    print(dict)
    return "produdct: 백설기 quantity: 2 price: 13,000"

## 주문 변경 처리 체인

In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a customer service assistant responsible for identifying and processing order changes based on customer chat history.
            First, find an AIMessage in the chat history that presents an 'order_id' and the corresponding order details, asking if the customer wishes to change the order.
            Then, look for the customer's response to the AIMessage regarding their desired changes. 
            Combine the AIMessage provided order details and the customer’s input to finalize the order changes.
            
            If the customer's response indicates that the presented order details are not the ones they want to change, do not proceed with the order change as it indicates a mismatch.
            """
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

In [24]:
order_change_chain = order_change_prompt | model
order_change_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a customer service assistant responsible for identifying and processing order changes based on customer chat history.\n            First, find an AIMessage in the chat history that presents an 'order_id' and the corresponding order details, asking if the customer wishes to change the order.\n            Then, look for the customer's response to the AIMessage regarding their desired changes. \n            Combine the AIMessage provided order details and the customer’s input to finalize the order changes.\n            \n            If the customer's response indicates that the presented order details are not the ones they want to change, do not proceed with the order change as it indicates a mismatch.\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatOpenAI(cli

In [25]:
from langchain_core.runnables.history import RunnableWithMessageHistory

order_change_chain_with_memory = RunnableWithMessageHistory(
    order_change_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [26]:
order_change_chain_with_memory_lambda = RunnableLambda(
    lambda x: order_change_chain_with_memory.invoke(x,
                                                    config={"configurable": {"session_id": "test_240517-3"}})
                                                    )

## 승인 메시지 포함 여부에 따른 라우트

In [27]:
def execution_or_message_route(info):
    print("execution_or_message_route 함수로 전달된 데이터 -> ", info)
    if "yes" in info["execution_confirmation"].content.lower():
        return order_change_chain_with_memory_lambda
    else:
        return RunnablePassthrough.assign(queried_result=fetch_recent_orders) | generate_confirm_message_chain_with_memory_lambda

In [28]:
execution_chain = confirmation_chain | execution_or_message_route
execution_chain

{
  inputs: RunnablePassthrough(),
  action_type: RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))
}
| RunnableLambda(helper)
| RunnableAssign(mapper={
    execution_confirmation: RunnableLambda(lambda x: classify_confirmation_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))
  })
| RunnableLambda(execution_or_message_route)

In [29]:
response = execution_chain.invoke(
    {"input": '없음',
     "order_id": 7}
)
response

Parent run 80a088b3-2460-40b9-82c9-448a83925449 not found for run 0b18b5e9-402f-4ef9-aead-003b84a40b03. Treating as a root run.
Parent run 452f7090-12c7-4747-b8ba-7fbb0a0ba883 not found for run 18c4317d-a1c0-49e7-98d6-917656b1ed1c. Treating as a root run.
Parent run fafef46f-83e6-4b59-b838-f655524815cd not found for run 210ecf68-28b3-412d-b85e-2c4c67e988a6. Treating as a root run.


execution_or_message_route 함수로 전달된 데이터 ->  {'input': '없음', 'order_id': 7, 'action_type': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 195, 'total_tokens': 198}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3ede3143-e0cd-4e95-aa7e-98eec2a2ae50-0'), 'execution_confirmation': AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 285, 'total_tokens': 286}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b2121e9d-45c7-47db-ac2f-ce8a9796806d-0')}
{'input': '없음', 'order_id': 7, 'action_type': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 195, 'total_tokens': 198}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3ede3143-e0cd-4e95-aa7e-98eec2a2ae5

AIMessage(content='주문 내용을 변경하려고 합니다.\n\n상품: 백설기\n수량: 2개\n가격: 13,000원\n\n변경 사항을 최종 확인해 주세요. 변경하시겠습니까?', response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 185, 'total_tokens': 253}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0dd09085-5b05-4aa4-96e8-a7ca10b0c0ae-0')

In [30]:
store["test_240517-3"].messages

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-22783fa0-2a50-4280-bd25-597866a41c1e-0'),
 HumanMessage(content='없음'),
 AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 195, 'total_tokens': 198}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3ede3143-e0cd-4e95-aa7e-98eec2a2ae50-0'),
 HumanMessage(content='없음'),
 AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 285, 'total_tokens': 286}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b2121e9d-45c7-47db-ac2f-ce8a9796806d-0'),
 HumanMessage(content='없음'),
 AIMessage(content='주문 내용을 변경하

In [33]:
response = execution_chain.invoke(
    {"input": '떡케익 13호로 바꿀게요',
     "order_id": 7}
)
response

Parent run 84fb21c8-b732-40e0-bed3-fecbd730b82d not found for run 637b7fb3-fd9a-499d-80b0-78e81141b19a. Treating as a root run.
Parent run 5a864ea0-7f5c-4d26-81a0-9271a8e8d78d not found for run 961ccf01-e051-4482-ab6b-98f0b8e0fd93. Treating as a root run.
Parent run 7e9f6317-84ee-4ab7-9476-d9ee7ac7af35 not found for run 11c14bbb-744c-4471-890c-5285dbc620c5. Treating as a root run.


execution_or_message_route 함수로 전달된 데이터 ->  {'input': '떡케익 13호로 바꿀게요', 'order_id': 7, 'action_type': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 315, 'total_tokens': 318}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f78aaf21-af09-4141-986a-64328a3c2150-0'), 'execution_confirmation': AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 418, 'total_tokens': 419}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-adddd686-4595-443e-a69e-ef341edb0974-0')}
{'input': '떡케익 13호로 바꿀게요', 'order_id': 7, 'action_type': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 315, 'total_tokens': 318}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f78aaf21-af09

AIMessage(content='주문을 변경하려고 합니다. 다음 내용으로 주문을 변경하시겠습니까?\n\n상품: 백설기\n수량: 2개\n가격: 13,000원\n\n최종 확정을 부탁드립니다.', response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 183, 'total_tokens': 258}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ff68b147-6d15-49e3-8649-75902bed8da5-0')

In [34]:
store["test_240517-3"].messages

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-22783fa0-2a50-4280-bd25-597866a41c1e-0'),
 HumanMessage(content='없음'),
 AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 195, 'total_tokens': 198}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3ede3143-e0cd-4e95-aa7e-98eec2a2ae50-0'),
 HumanMessage(content='없음'),
 AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 285, 'total_tokens': 286}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b2121e9d-45c7-47db-ac2f-ce8a9796806d-0'),
 HumanMessage(content='없음'),
 AIMessage(content='주문 내용을 변경하